In [112]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import re
import requests
import unidecode
import string
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')


In [3]:
hilary = pd.read_csv('data/hilary_scores.csv')

In [110]:
unique_url = hilary['URL'].unique()
len(unique_url)

110

In [111]:
# url = 'http://www.nytimes.com/2015/07/16/world/middleeast/obama-diplomacy-iran-nuclear-deal.html'
# table = hilary[['Comment', 'URL']][hilary['URL'] == url]
# table

In [6]:
## strip HTML tags
def remove_html_tags(data):
    p = re.compile(r'<[^<]*?>')
    return p.sub('', data)

remove_html_tags(x)

'Why America Is Not the Greatest Country in the World!YouTube:" The Newsroom "America is not the greatest country in the world anymore" 2012 TV series "'

In [ ]:
all_body = []
for url in unique_url:
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find_all('p', class_= "story-body-text story-content")
    content = '\n'.join([i.text for i in soup.select('p.story-body-text')])
    all_body.append(unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t",""))

documents = hilary['Comment']
test = np.array(body_text)
documents = np.append(documents.values, test)

In [70]:
#get body text
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
body = soup.find_all('p', class_= "story-body-text story-content")

content = '\n'.join([i.text for i in soup.select('p.story-body-text')])
body_text = unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t","")


In [84]:
test = np.array(body_text)
documents = np.append(documents.values, test)

In [86]:
def tokenize(doc):
    return [snowball.stem(word) for word in word_tokenize(doc.lower())]

In [87]:
tfidfvect = TfidfVectorizer(stop_words='english', tokenizer=tokenize)


In [90]:
# vect = CountVectorizer(stop_words='english')
# word_counts = vect.fit_transform(documents)
tfidf_vectorized = tfidfvect.transform(documents).toarray()
cosine_similarities = []

for doc in documents:
    cosine_similarities.append(linear_kernel(tfidf_vectorized, tfidf_vectorized))

In [99]:
#similarity between article and each of the comment
cosine_similarities[0][25]
m = np.array(cosine_similarities[0][25])
m = m[:25]
table["Similarity"] = m
table.head()

array([ 0.23870678,  0.24296168,  0.20111041,  0.19704109,  0.22639313,
        0.19200265,  0.15639517,  0.35807008,  0.26233613,  0.25164245,
        0.18980466,  0.30549497,  0.02907837,  0.16125712,  0.06182941,
        0.28607225,  0.15076563,  0.16433662,  0.12515468,  0.30239537,
        0.07784808,  0.1918442 ,  0.20046098,  0.33353429,  0.35674505,  1.        ])

,Comment,URL,Similarity
313,Obama and the Democrats need to remind the pub...,http://www.nytimes.com/2015/07/16/world/middle...,0.238707
314,Iran realizes that having a bomb is really not...,http://www.nytimes.com/2015/07/16/world/middle...,0.242962
315,After listening on C-Span to the nuclear exper...,http://www.nytimes.com/2015/07/16/world/middle...,0.201110
316,The key question is. How effective are the me...,http://www.nytimes.com/2015/07/16/world/middle...,0.197041
317,"The reality, Iran is and will continue to be a...",http://www.nytimes.com/2015/07/16/world/middle...,0.226393
